In [5]:
# set env variable to json credential file of google cloud

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/karthikmahendra/Desktop/Salty/salty-trolls-cce24104ff9f.json'

In [6]:
from google.cloud import bigquery
import time
import pandas as pd
import numpy as np

In [7]:
# setup client
bq_client = bigquery.Client() 

In [8]:
#Query for pulling data of a username
QUERY = '''SELECT  * 
        FROM `bigquery-public-data.hacker_news.full_201510`
        WHERE `bigquery-public-data.hacker_news.full_201510`.by = "danmaz74"
        '''

In [9]:
df = bq_client.query(QUERY).to_dataframe()

In [10]:
df.shape

(1680, 13)

In [12]:
df['text'] = df['text'].str.encode('ascii', 'ignore').str.decode('ascii')

In [13]:
df.head()

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,danmaz74,NaN,1368003409,None,comment,None,"""HackingItalia"" has much better chances of suc...",5672999.0,None,None,NaN,5673213,None
1,danmaz74,NaN,1362914038,None,comment,None,The (not so) funny thing is that this has been...,5350893.0,None,None,NaN,5351456,None
2,danmaz74,NaN,1380696100,None,comment,None,Absolutely. There is also no clue that the pag...,6479777.0,None,None,NaN,6481042,None
3,danmaz74,NaN,1370794029,None,comment,None,Does anybody have ideas about how a small fore...,5849087.0,None,None,NaN,5849372,None
4,danmaz74,NaN,1383740372,None,comment,None,"Great, I&#x27;ll try your service instead of d...",6681977.0,None,None,NaN,6682294,None


In [17]:
import html,re

In [18]:
def decoder(text):
    """decode HTML text."""
    unescaped = html.unescape(text)
    return unescaped


def kill_html(text):
    """Remove html flags."""
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', text)
    return cleantext


def kill_url(text):
    """Remove URL with regex"""
    return ''.join(re.sub(r"http\S+", "", text))



df['cleaned_comment'] = df.text.apply(lambda x: kill_url(kill_html(decoder(x))))

TypeError: argument of type 'NoneType' is not iterable

In [24]:
# job_config = bigquery.QueryJobConfig()
# now=time.time()
# query_job=bq_client.query(QUERY,location='US')
# res=query_job.result()
# print('query took:',round(time.time()-now,2),'s')

query took: 1.23 s


In [13]:
# now=time.time()
# destination_uri = "gs://karthikkm028-ds/*hacker.csv"
# dataset_ref = bq_client.dataset("hacker_news", project='bigquery-public-data')
# table_ref = dataset_ref.table("full_201510")

# extract_job = bq_client.extract_table(
#     table_ref,
#     destination_uri)
# extract_job.result()  # Waits for job to complete
# print('create table and write to GCS took:',round(time.time()-now,2),'s')

In [60]:
# gcs = gcsfs.GCSFileSystem(project='bigquery-public-data') 
# files=gcs.glob(destination_uri)
# df = pd.concat([pd.read_csv('gs://'+f) for f in files], ignore_index=True)
# print('read csv took:',round(time.time()-now,2),'s')

/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (1,2,5,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


read csv took: 1421.34 s


In [101]:

df_drop = df.drop(columns= ['score','title','url','deleted','dead','descendants','ranking'])

In [102]:
df_drop.head()

,by,time,type,text,parent,id
0,danmaz74,1406974965,comment,&quot;Adapter&quot;?,8124494.0,8124572
1,danmaz74,1436624566,comment,"For the Romans, an often cited reason for the ...",9868765.0,9869850
2,danmaz74,1338129641,comment,And that's all I need ;),4030140.0,4030164
3,danmaz74,1421700122,comment,"With a couple of books I reference often, I&#x...",8913265.0,8913948
4,danmaz74,1352372488,comment,"On the other hand, 99.7% of my readers didn't ...",4757225.0,4757283


In [103]:
df_drop.shape

(1680, 6)

In [104]:
df_drop.dtypes

by         object
time        int64
type       object
text       object
parent    float64
id          int64
dtype: object

In [61]:
#df1 = df_drop.head(100)

In [105]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [198]:
df_drop['text']=df_drop['text'].apply(str)

In [200]:
df_drop['text']=df_drop['text'].str.encode('ascii',errors='ignore')

In [107]:
type(df_drop['text'][0])

str

In [108]:
s = sia.polarity_scores(df_drop['text'][0])

In [109]:
s

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [110]:
s['neg']

0.0

In [113]:
#df_drop['neg'][0] = sia.polarity_scores(df_drop['text'][0])['neg']

In [116]:
sia.polarity_scores(df_drop['text'][0])

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [129]:
df_drop['neg']=np.zeros(df_drop.shape[0])
df_drop['pos']=np.zeros(df_drop.shape[0])
df_drop['neu']=np.zeros(df_drop.shape[0])

In [130]:
# populate vader sentiments in additional cols
for idx,x in df_drop['text'].iteritems():
    #print (idx,x)
    df_drop['neg'][idx] = sia.polarity_scores(x)['neg']
    df_drop['pos'][idx] = sia.polarity_scores(x)['pos']
    df_drop['neu'][idx]= sia.polarity_scores(x)['neu']


/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/karthikmahendra/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [201]:
df_drop.head()

,by,time,type,text,parent,id,neg,pos,neu
0,danmaz74,1406974965,comment,b'&quot;Adapter&quot;?',8124494.0,8124572,0.000,0.000,1.000
1,danmaz74,1436624566,comment,"b'For the Romans, an often cited reason for th...",9868765.0,9869850,0.080,0.000,0.920
2,danmaz74,1338129641,comment,"b""And that's all I need ;)""",4030140.0,4030164,0.000,0.322,0.678
3,danmaz74,1421700122,comment,"b'With a couple of books I reference often, I&...",8913265.0,8913948,0.000,0.087,0.913
4,danmaz74,1352372488,comment,"b""On the other hand, 99.7% of my readers didn'...",4757225.0,4757283,0.053,0.138,0.810


In [161]:
# Set threshold of .5 for filtering positive and negative comments. drop comments with value 1 

df_neg = df_drop[(df_drop['neg']> 0.5) & (df_drop['neg'] !=1)]
df_pos = df_drop[(df_drop['pos']> 0.5) & (df_drop['pos'] !=1)]

In [162]:
df_neg = df_neg.sort_values(['neg'],ascending=False)
df_pos =df_pos.sort_values(['pos'],ascending=False)

In [202]:
# top negative comments
df_neg.head()

,by,time,type,text,parent,id,neg,pos,neu
1345,danmaz74,1375567037,comment,"I&#x27;m disgusted, this is such a rigged game.",6152872.0,6153574,0.541,0.0,0.459
434,danmaz74,1431625440,comment,They lost us...,9546379.0,9546430,0.535,0.0,0.465


In [186]:
# top positive comments
df_pos.head()

,by,time,type,text,parent,id,neg,pos,neu
414,danmaz74,1441294856,comment,"Thanks, great answer!",10165509.0,10165541,0.0,0.879,0.121
699,danmaz74,1318260807,comment,Indeed... good luck!,3094081.0,3094183,0.0,0.861,0.139
1244,danmaz74,1379140076,comment,"Yes - yes - camera button, please!!",6384270.0,6384874,0.0,0.806,0.194
14,danmaz74,1438182885,comment,You&#x27;re welcome!,9969067.0,9969154,0.0,0.767,0.233
510,danmaz74,1368988078,comment,Exactly :),5733149.0,5733876,0.0,0.750,0.250
589,danmaz74,1320396226,comment,Good luck with it then :),3193271.0,3195378,0.0,0.748,0.252
1218,danmaz74,1352378421,comment,"Ok, I'll look for a solution. Thanks!",4757386.0,4757574,0.0,0.719,0.281
963,danmaz74,1429266929,comment,Good job David ;),9391828.0,9393355,0.0,0.706,0.294
1078,danmaz74,1418308852,comment,Font awesome has spinners... :),8734901.0,8734915,0.0,0.703,0.297
1651,danmaz74,1338135787,comment,Cool - so for FF there is already a solution :),4030324.0,4030348,0.0,0.695,0.305


In [192]:
from datetime import datetime

#convert 

df_pos['time']=pd.to_datetime(df_pos['time'],unit='s')

In [195]:
df['time']=pd.to_datetime(df['time'],unit='s')

In [196]:
s = s.encode('ascii',errors='ignore')

,by,score,time,title,type,url,text,parent,deleted,dead,descendants,id,ranking
0,danmaz74,NaN,2014-08-02 10:22:45,None,comment,None,&quot;Adapter&quot;?,8124494.0,None,None,NaN,8124572,None
1,danmaz74,NaN,2015-07-11 14:22:46,None,comment,None,"For the Romans, an often cited reason for the ...",9868765.0,None,None,NaN,9869850,None
2,danmaz74,NaN,2012-05-27 14:40:41,None,comment,None,And that's all I need ;),4030140.0,None,None,NaN,4030164,None
3,danmaz74,NaN,2015-01-19 20:42:02,None,comment,None,"With a couple of books I reference often, I&#x...",8913265.0,None,None,NaN,8913948,None
4,danmaz74,NaN,2012-11-08 11:01:28,None,comment,None,"On the other hand, 99.7% of my readers didn't ...",4757225.0,None,None,NaN,4757283,None
5,danmaz74,NaN,2015-06-23 19:07:54,None,comment,None,"I&#x27;m not against nuclear, but also conside...",9765836.0,None,None,NaN,9766956,None
6,danmaz74,NaN,2013-10-17 10:28:12,None,comment,None,&gt; Companies are for making money<p>That&#x2...,6564375.0,None,None,NaN,6564467,None
7,danmaz74,NaN,2012-11-10 19:35:09,None,comment,None,"Italy is pretty ""private"" about this. I person...",4767420.0,None,None,NaN,4767492,None
8,danmaz74,NaN,2013-06-25 14:20:07,None,comment,None,&quot;someone who turned because of what he sa...,5939415.0,None,None,NaN,5939584,None
9,danmaz74,NaN,2012-10-10 11:43:13,None,comment,None,You're welcome!<p>I didn't think about the cod...,4635880.0,None,None,NaN,4635893,None
